# Complex SBI Model Training

In this example we will demonstrate how to customise the training of your SBI model. In the [Basic SBI Training](basic_sbi_model.ipynb) tutorial we trained a default model, but here we will explore some of the cutomisation options available to you.

In [ ]:
from synference import SBI_Fitter

fitter = SBI_Fitter.init_from_hdf5(model_name="test", hdf5_path="test_model_grid.hdf5")

We'll use the default feature array settings for this example, but you can refer to the [Feature Arrays](feature_array.ipynb) tutorial to learn how to create your own custom feature arrays.


In [ ]:
fitter.create_feature_array();

Let's look at the options available in the `run_single_sbi` method.

In [ ]:
?fitter.run_single_sbi

We will break down the options into a few categories:
1. **Model Architecture Options**: These options allow you to customize the architecture of the neural network used in the SBI model.
- _backend_: This lets you choose between the SBI backends supported by LtU-ILI. The default is `sbi`, but you can also choose `lampe` if you prefer that framework.
- _engine_: This option allows you to select the inference engine used for training. The default is `NPE`, but you can also choose `NLE` or `NRE` depending on your needs. Sequentual methods are also supported (SNPE, SNLE, SNRE). 
- _model_type_: This option lets you specify the type of model to use. The default is `mdn` (Mixture Density Network), but you can also choose `maf` (Masked Autoregressive Flow) or `nsf` (Neural Spline Flow) for more complex distributions. See the documentation for sbi or lampe for the full list of supported model types.
- _hidden_features_: This option allows you to set the number of hidden features (neurons) in each layer of the neural network. The default is 50, but you can increase this for more complex models.
- _num_components_: This option lets you specify the number of mixture components in the Mixture Density Network. The default is 4, but you can increase this for more complex distributions. 
- _num_transforms_: This option allows you to set the number of transforms in normalizing flow models (like MAF or NSF). The default is 4, but you can increase this for more complex distributions.
- _embedding_net_: This option lets you provide a custom embedding network for processing the observations. By default no embedding network is used, but you can create your own using PyTorch and pass it here.
- _n_nets_: This option allows you to specify the number of networks to use in an ensemble. The default is 1, but you can increase this to train an ensemble of models which may improve performance. Model parameters such as _num_components_ and _hidden_features_ can be specified as lists to vary them across the ensemble members.

2. **Training Options**: These options allow you to customize the training process of the SBI model.
- _train_test_fraction_: This option lets you specify the fraction of data to use for training versus testing. The default is 0.8 (80% training, 20% testing), but you can adjust this based on your dataset size.
- _training_batch_size_: This option allows you to set the batch size for training. The default is 64, but you can vary this based on your hardware capabilities.
- _learning_rate_: This option lets you specify the learning rate for the optimizer. The default is 1e-4, but you can adjust this based on your training dynamics.
- _override_prior_ranges_: This option allows you to override the prior ranges defined in the model grid. You can provide a dictionary with parameter names as keys and tuples of (min, max) as values to set new prior ranges. The default parameter ranges are the minimum and maximum values defined in the model grid.
- _use_existing_indices_: This option lets you choose whether to use existing training and testing indices if they are already saved on the object. The default is `True`, which will reuse the existing indices to ensure consistency across runs.
- _validation_fraction_: This option allows you to specify the fraction of training data to use for validation during training. The default is 0.1 (10% of training data), but you can adjust this based on your needs.
- _stop_after_epochs_: The number of epochs without improvement on the validation set after which training will stop early. The default is 15, but you can adjust this based on your training dynamics.
- _train_indices_/_test_indices_: These options allow you to provide custom indices for training and testing data. By default, random indices are generated based on the _train_test_fraction_.

3. **Miscellaneous Options**: These options provide additional customization for the SBI model training.
- _load_existing_model_: This option allows you to load an existing trained model instead of training a new one if the named model file is found. The default is `True`.
- _evaluate_model_: This option lets you choose whether to evaluate the model after training. The default is `True`, which will compute metrics on the test set.
- _save_model_: This option allows you to save the trained model to disk. The default is `True`.
- _name_append_: This option lets you append a custom string to the model name for easier identification. The default is a timestamp.
- _set_self_: This option will set the trained model and various attributes to the current SBI object. The default is `True`.
- _plot_: This option allows you to generate plots of the training results. The default is `True`.
- _verbose_: This option lets you control the verbosity of the training output. The default is `True`, which will print detailed information during training.
- _save_method_: This option allows you to specify the method for saving the model. The default is `joblib`, but you can choose other methods supported by the backend.
- _num_posterior_draws_per_sample_: This option lets you specify the number of posterior draws to make per sample during evaluation. The default is 1000.
- _random_seed_: This option allows you to set a random seed for reproducibility. The default is `None`, which means no specific seed is set.
- _out_dir_: This option lets you specify the output directory where models and results will be saved. The default is the models/ folder.
4. **Online Training Options**: These options allow you to customize the online training process of the SBI model.
- _learning_type_: This option lets you specify the type of learning to perform. For online training, set this to `online`. The default is `amortized`.
- _num_online_rounds_: This option lets you specify the number of online training rounds to perform. The default is 5, but you can adjust this based on your needs.
- _num_simulations_: The number of simulations to run in each online training round. The default is 1000, but you can adjust this based on your computational resources.
- _online_training_xobs_: As online training is not amortized, you need to provide the specific observation(s) for which you want to perform inference during online training. This option allows you to specify those observations.
- _initial_training_from_grid_: This option lets you specify whether to perform an initial training phase using a grid of simulations. The default is `False`.
-simulator_: This option allows you to provide a custom simulator function for generating simulations during online training. By default, the simulator is attempted to be retrieved from the model grid.

Here's an example where we use the lampe backend to train an ensemble of 2 models using different architectures (NSF and MAF) with varying hidden features and number of transforms. We also adjust the training parameters such as train-test split, batch size, and learning rate. 

In [ ]:
fitter.run_single_sbi(
    backend="lampe",
    engine="NPE",
    model_type=["nsf", "maf"],
    hidden_features=[30, 40],
    num_transforms=[18, 36],
    n_nets=2,
    train_test_fraction=0.9,
    training_batch_size=128,
    learning_rate=5e-4,
    override_prior_ranges={"log_mass": [7.75, 12.25]},
);